In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, ReLU, MaxPooling2D, SeparableConv2D, Dense, Concatenate, BatchNormalization, GlobalAveragePooling2D, Add, AveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
import os

In [10]:
def load_dataset(dataset_path, image_size=(120, 120)):
    images = []
    labels = []
    class_names = ['Artificial', 'Natural']

    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        
        for file in os.listdir(class_dir):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
                img_path = os.path.join(class_dir, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img)
                img_array /= 255.0
                images.append(img_array)
                
                # Append the label to the labels list as a one-hot encoded array
                label = to_categorical(class_names.index(class_name), num_classes=2)
                labels.append(np.expand_dims(np.stack([label, 1 - label]), axis=0))  # Stack label and 1-label

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Example usage:
dataset_path = 'Dataset_7'
images, labels = load_dataset(dataset_path)

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into a training set and a hold-out test set
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.25, random_state=42
)

In [12]:
# Define the number of splits for k-fold cross-validation
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [13]:
def create_head_model():
    input_layer = Input(shape=(120, 120, 3))
    # First block
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(input_layer)
    x = ReLU()(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = ReLU()(x)
    
    x1 = x
    x1 = Conv2D(128, (1, 1), strides=(2, 2))(x1)

    # Second block
    x = SeparableConv2D(128, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = SeparableConv2D(128, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    x = tf.add(x, x1)

    x2 = x
    x2 = Conv2D(256, (1, 1), strides=(2, 2))(x2)

    # Third block
    x = SeparableConv2D(256, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = SeparableConv2D(256, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    x = tf.add(x, x2)

    x3 = x
    x3 = Conv2D(728, (1, 1), strides=(2, 2))(x3)

    # Fourth block
    x = SeparableConv2D(728, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = SeparableConv2D(728, (3, 3), padding='same')(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    x = tf.add(x, x3)
    
    # Middle Flow (repeated block)
    for _ in range(2):
        block_input = x
        x = ReLU()(x)
        x = SeparableConv2D(728, (3, 3), padding='same')(x)
        x = ReLU()(x)
        x = SeparableConv2D(728, (3, 3), padding='same')(x)
        x = ReLU()(x)
        x = SeparableConv2D(728, (3, 3), padding='same')(x)
        # Add the block_input (residual connection), not visible in the diagram
        x = tf.add(x, block_input)
    
    x = ReLU()(x)
    head_model = Model(inputs=input_layer, outputs=x)
    return head_model

In [14]:
def create_ensemble_model(head_model, num_submodels):
    inputs = Input(shape=(120, 120, 3))
    head_outputs = head_model(inputs)

    submodel_outputs = []
    for i in range(num_submodels):
        x = SeparableConv2D(4, (3, 3), strides=(2, 2), padding='same')(head_outputs)
        x = ReLU()(x)
        x = SeparableConv2D(16, (3, 3), strides=(2, 2), padding='same')(x)
        x = ReLU()(x)
        x = Dense(1, activation='sigmoid', name=f'output_{i}')(x)  # Sigmoid activation for binary classification
        submodel_outputs.append(x)
    
    if len(submodel_outputs) > 1:
        ensemble_outputs = Concatenate()(submodel_outputs)
    else:
        ensemble_outputs = submodel_outputs[0]

    ensemble_model = Model(inputs=inputs, outputs=ensemble_outputs)
    return ensemble_model

In [15]:
# Instantiate the head model
head_model = create_head_model()

# Define the number of submodels in the ensemble
num_submodels = 2  # Replace with the number of submodels you have

# Instantiate the ensemble model
ensemble_model = create_ensemble_model(head_model, num_submodels)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# K-fold Cross Validation model evaluation

ensemble_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

kf = KFold(n_splits=4, shuffle=True, random_state=42)

fold_no = 1
for train_index, val_index in kf.split(train_images):
    smaller_train_images, val_images = train_images[train_index], train_images[val_index]
    smaller_train_labels, val_labels = train_labels[train_index], train_labels[val_index]

    history = ensemble_model.fit(
        smaller_train_images, smaller_train_labels,
        batch_size=32,
        epochs=10,
        validation_data=(val_images, val_labels)
    )

    fold_no += 1

Epoch 1/10
109/109 [==============================] - 70s 616ms/step - loss: 0.6894 - accuracy: 0.6485 - val_loss: 0.6744 - val_accuracy: 0.6637
Epoch 2/10
109/109 [==============================] - 75s 692ms/step - loss: 0.6676 - accuracy: 0.6526 - val_loss: 0.6484 - val_accuracy: 0.6646
Epoch 3/10
109/109 [==============================] - 79s 727ms/step - loss: 0.6520 - accuracy: 0.6527 - val_loss: 0.6406 - val_accuracy: 0.6650
Epoch 4/10
 56/109 [==============>...............] - ETA: 38s - loss: 0.6433 - accuracy: 0.6567